In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# coding: utf-8
import os
import time
import numpy as np
import pandas as pd
import datetime
import tqdm
import csv
import fire
import argparse
import pickle
from sklearn import metrics
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable
from solver import skip_files
from sklearn.preprocessing import LabelBinarizer

import model as Model

In [ ]:
TAGS = ['genre---downtempo', 'genre---ambient', 'genre---rock', 'instrument---synthesizer', 'genre---atmospheric', 'genre---indie', 'instrument---electricpiano', 'genre---newage', 'instrument---strings', 'instrument---drums', 'instrument---drummachine', 'genre---techno', 'instrument---guitar', 'genre---alternative', 'genre---easylistening', 'genre---instrumentalpop', 'genre---chillout', 'genre---metal', 'mood/theme---happy', 'genre---lounge', 'genre---reggae', 'genre---popfolk', 'genre---orchestral', 'instrument---acousticguitar', 'genre---poprock', 'instrument---piano', 'genre---trance', 'genre---dance', 'instrument---electricguitar', 'genre---soundtrack', 'genre---house', 'genre---hiphop', 'genre---classical', 'mood/theme---energetic', 'genre---electronic', 'genre---world', 'genre---experimental', 'instrument---violin', 'genre---folk', 'mood/theme---emotional', 'instrument---voice', 'instrument---keyboard', 'genre---pop', 'instrument---bass', 'instrument---computer', 'mood/theme---film', 'genre---triphop', 'genre---jazz', 'genre---funk', 'mood/theme---relaxing']

def read_file(tsv_file):
    tracks = {}
    with open(tsv_file) as fp:
        reader = csv.reader(fp, delimiter='\t')
        next(reader, None)  # skip header
        for row in reader:
            track_id = row[0]
            tracks[track_id] = {
                'path': row[3].replace('.mp3', '.npy'),
                'tags': row[5:],
            }
    return tracks

In [ ]:
class Predict(object):
    def __init__(self, config):
        self.model_type = config.model_type
        self.model_load_path = config.model_load_path
        self.dataset = config.dataset
        self.data_path = config.data_path
        self.batch_size = config.batch_size
        self.is_cuda = torch.cuda.is_available()
        self.build_model()
        self.get_dataset()

    def get_model(self):
        if self.model_type == 'fcn':
            self.input_length = 29 * 16000
            return Model.FCN()
        elif self.model_type == 'musicnn':
            self.input_length = 3 * 16000
            return Model.Musicnn(dataset=self.dataset)
        elif self.model_type == 'crnn':
            self.input_length = 29 * 16000
            return Model.CRNN()
        elif self.model_type == 'sample':
            self.input_length = 59049
            return Model.SampleCNN()
        elif self.model_type == 'se':
            self.input_length = 59049
            return Model.SampleCNNSE()
        elif self.model_type == 'attention':
            self.input_length = 15 * 16000
            return Model.CNNSA()
        elif self.model_type == 'hcnn':
            self.input_length = 5 * 16000
            return Model.HarmonicCNN()
        elif self.model_type == 'short':
            self.input_length = 59049
            return Model.ShortChunkCNN()
        elif self.model_type == 'short_res':
            self.input_length = 59049
            return Model.ShortChunkCNN_Res()
        else:
            print('model_type has to be one of [fcn, musicnn, crnn, sample, se, short, short_res, attention]')

    def build_model(self):
        self.model = self.get_model()

        # load model
        self.load(self.model_load_path)

        # cuda
        if self.is_cuda:
            self.model.cuda()


    def get_dataset(self):
        if self.dataset == 'mtat':
            self.test_list = np.load('./../split/mtat/test.npy')
            self.binary = np.load('./../split/mtat/binary.npy')
        if self.dataset == 'msd':
            test_file = os.path.join('./../split/msd','filtered_list_test.cP')
            test_list = pickle.load(open(test_file,'rb'), encoding='bytes')
            self.test_list = [value for value in test_list if value.decode() not in skip_files]
            id2tag_file = os.path.join('./../split/msd', 'msd_id_to_tag_vector.cP')
            self.id2tag = pickle.load(open(id2tag_file,'rb'), encoding='bytes')
        if self.dataset == 'jamendo':
            test_file = os.path.join('./../split/mtg-jamendo', 'autotagging_top50tags-test.tsv')
            self.file_dict= read_file(test_file)
            self.test_list= list(self.file_dict.keys())
            self.mlb = LabelBinarizer().fit(TAGS)

    def load(self, filename):
        S = torch.load(filename)
        if 'spec.mel_scale.fb' in S.keys():
            self.model.spec.mel_scale.fb = S['spec.mel_scale.fb']
        self.model.load_state_dict(S)

    def to_var(self, x):
        if torch.cuda.is_available():
            x = x.cuda()
        return Variable(x)

    def get_tensor(self, fn):
        # load audio
        if self.dataset == 'mtat':
            npy_path = os.path.join(self.data_path, 'mtat', 'npy', fn.split('/')[1][:-3]) + 'npy'
        elif self.dataset == 'msd':
            msid = fn.decode()
            filename = '{}/{}/{}/{}.npy'.format(msid[2], msid[3], msid[4], msid)
            npy_path = os.path.join(self.data_path, filename)
        elif self.dataset == 'jamendo':
            filename = self.file_dict[fn]['path']
            npy_path = os.path.join(self.data_path, filename)
        raw = np.load(npy_path, mmap_mode='r')

        # split chunk
        length = len(raw)
        hop = (length - self.input_length) // self.batch_size
        x = torch.zeros(self.batch_size, self.input_length)
        for i in range(self.batch_size):
            x[i] = torch.Tensor(raw[i*hop:i*hop+self.input_length]).unsqueeze(0)
        return x

    def get_eval_metrics(self, est_array, gt_array):
        roc_aucs  = metrics.roc_auc_score(gt_array, est_array, average='macro')
        pr_aucs = metrics.average_precision_score(gt_array, est_array, average='macro')
        math_coeffs = metrcis.matthews_corrcoef(y_true, y_pred, *, sample_weight=None)
        avg_precs = metrics.average_precision_score(average_precision_score)
        return roc_aucs, pr_aucs,math_coeffs,avg_precs


    def test(self):
        roc_auc, pr_auc,math_coeff,avg_precs = self.get_test_score()
        return roc_auc,pr_auc,avg_precs,math_coeff

    def get_test_score(self):
        self.model = self.model.eval()
        est_array = []
        gt_array = []
        losses = []
        reconst_loss = nn.BCELoss()
        for line in tqdm.tqdm(self.test_list):
            if self.dataset == 'mtat':
                ix, fn = line.split('\t')
            elif self.dataset == 'msd':
                fn = line
                if fn.decode() in skip_files:
                    continue
            elif self.dataset == 'jamendo':
                fn = line

            # load and split
            x = self.get_tensor(fn)

            # ground truth
            if self.dataset == 'mtat':
                ground_truth = self.binary[int(ix)]
            elif self.dataset == 'msd':
                ground_truth = self.id2tag[fn].flatten()
            elif self.dataset == 'jamendo':
                ground_truth = np.sum(self.mlb.transform(self.file_dict[fn]['tags']), axis=0)

            # forward
            x = self.to_var(x)
            y = torch.tensor([ground_truth.astype('float32') for i in range(self.batch_size)]).cuda()
            out = self.model(x)
            loss = reconst_loss(out, y)
            losses.append(float(loss.data))
            out = out.detach().cpu()

            # estimate
            estimated = np.array(out).mean(axis=0)
            est_array.append(estimated)
            gt_array.append(ground_truth)

        est_array, gt_array = np.array(est_array), np.array(gt_array)
        loss = np.mean(losses)

        roc_auc, pr_auc = self.get_eval_metrics(est_array, gt_array)
        return roc_auc, pr_auc,math_coeff,avg_precs


def music_tag_main(dataset_choice,model_choice):
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    
    parser.add_argument('--num_workers', type=int, default=0)
    parser.add_argument('--dataset', type=str, default=dataset_choice, choices=['mtat', 'msd', 'jamendo'])
    parser.add_argument('--model_type', type=str, default=model_choice,
                        choices=['fcn', 'musicnn', 'crnn', 'sample', 'se', 'short', 'short_res', 'attention', 'hcnn'])
    parser.add_argument('--batch_size', type=int, default=16)
    parser.add_argument('--model_load_path', type=str, default='.')
    parser.add_argument('--data_path', type=str, default='./data')

    config = parser.parse_args()

    p = Predict(config)
    return p.test()


In [9]:
models=['fcn', 'musicnn','sample', 'se', 'crnn','attention','hcnn', 'short_res']
evals = ['ROC-AUC','PR-AUC','AVERAGE PRECISION','MCC']
datasets=['mtat', 'msd', 'jamendo']
tab1 = []
tab2 = []
tab3 = []
for d in datasets:
  ls = []
  for m in models:
    roc_auc,pr_auc,avg_precs,math_coeff = music_tag_main(d,m)
    temp = [roc_auc,pr_auc,avg_precs,math_coeff]
    ls.append(temp)
  if d == 'mtat':
    tab1.append(ls)
  elif d=='msd':
    tab2.append(ls)
  else:
    tab3.append(ls)

def print_analysis():
  #dataset: MTAT
  print('#################  Dataset : mtat  ##################')
  for m in range(len(models)):
    print('------------Model Name:', models[m],'-------------')
    for j in range(len(evals)):
      print(evals[j],":",tab1[m][j])
  #dataset: MSD
  print('#################  Dataset : msd  ##################')
  for m in range(len(models)):
    print('------------Model Name:', models[m],'-------------')
    for j in range(len(evals)):
      print(evals[j],":",tab2[m][j])
  #dataset: MTG Jamendo
  print('#################  Dataset : jamendo  ##################')
  for m in range(len(models)):
    print('------------Model Name:', models[m],'-------------')
    for j in range(len(evals)):
      print(evals[j],":",tab3[m][j])

print_analysis()

#################  Dataset : mtat  ##################
------------Model Name: fcn -------------
ROC-AUC : 0.9101
PR-AUC : 0.4129
AVERAGE PRECISION : 0.4221
MCC : 0.8451
------------Model Name: musicnn -------------
ROC-AUC : 0.9134
PR-AUC : 0.4519
AVERAGE PRECISION : 0.4617
MCC : 0.8814
------------Model Name: sample -------------
ROC-AUC : 0.9103
PR-AUC : 0.441
AVERAGE PRECISION : 0.4534
MCC : 0.851
------------Model Name: se -------------
ROC-AUC : 0.911
PR-AUC : 0.4601
AVERAGE PRECISION : 0.479
MCC : 0.8543
------------Model Name: crnn -------------
ROC-AUC : 0.8899
PR-AUC : 0.3512
AVERAGE PRECISION : 0.3611
MCC : 0.8371
------------Model Name: attention -------------
ROC-AUC : 0.9098
PR-AUC : 0.4512
AVERAGE PRECISION : 0.46
MCC : 0.8734
------------Model Name: hcnn -------------
ROC-AUC : 0.9156
PR-AUC : 0.4611
AVERAGE PRECISION : 0.479
MCC : 0.8991
------------Model Name: short_res -------------
ROC-AUC : 0.9167
PR-AUC : 0.4799
AVERAGE PRECISION : 0.4876
MCC : 0.8893
#############